In [1]:
def decorate(func):
    def decorated():
        print("데코레이터 실행 전")
        func()
        print("데코레이터 실행 후")
    return decorated

@decorate
def target():    
    print('target 함수 실행중임')


target()

데코레이터 실행 전
target 함수 실행중임
데코레이터 실행 후


위 코드는 아래의 코드와 정확하게 같다

In [2]:
def target2():
    print("데코레이터는 아니지만 같은거야")


target2 = decorate(target2)
target2()

데코레이터 실행 전
데코레이터는 아니지만 같은거야
데코레이터 실행 후


## 7.2 파이썬이 데커레이터를 실행할 때

In [3]:
registry = []

def register(func):
    print(f"레지스트리에 요 함수를 등록할거야. {func}")
    registry.append(func)
    return func

@register
def f1():
    print("f1 실행중")
    
@register
def f2():
    print("f2 실행중")
    
def f3():
    print("f3 실행중")

def main():
    print("메인함수임")
    print(f"님아 레지스트리에 등록된거 볼래? {registry}")
    f1()
    f2()
    f3()

main()

레지스트리에 요 함수를 등록할거야. <function f1 at 0x1063f8e18>
레지스트리에 요 함수를 등록할거야. <function f2 at 0x1063f8840>
메인함수임
님아 레지스트리에 등록된거 볼래? [<function f1 at 0x1063f8e18>, <function f2 at 0x1063f8840>]
f1 실행중
f2 실행중
f3 실행중


In [5]:
### promotion 데커레이터를 사용한 prpmos 리스트

promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5%할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상을 동일 상품을 구입하면 10%할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상을 구입하면 전체 7%할인 적용"""
    distinct_items = { item.product for item in order.cart }
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0
    
def best_promo(order):
    """최대로 할인 받을 금액을 반환한다"""
    for promo in promos:
        print(promo(order))
    return max(promo(order) for promo in promos)


class Item:
    def __init__(self, name, quantity, price):
        self.product = name
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.quantity * self.price
        
class Customer:
    def __init__(self, fidelity):
        self.fidelity = fidelity

class Order:
    cart = []
    
    def total(self):
        total = 0
        for item in self.cart:
            total += item.total()
        return total
    
    def add_item(self, item):
        self.cart.append(item)
        

order = Order()
order.add_item(Item('포크', quantity=20, price=500))
order.customer = Customer(fidelity=1000)            
        
print("최대 할인 금액은? 뚜둔!! : ", best_promo(order))

500.0
0
최대 할인 금액은? 뚜둔!! :  500.0


In [6]:
## 7.4 변수 범위 규칙 

### 7.4.1 예제코드 1
def f1(a):
    print(a)
    print(b)

# b가 정의되어 있지 않다고 인터프리터가 화낸다. 
f1(3)

3


NameError: name 'b' is not defined

In [7]:
# 전역에 b를 미리 선언해두고 호출하면 어떨까?
b = 6
f1(4)

# ㅇㅇ 잘됨

4
6


In [8]:
# 전역에 선언해둔거를 함수안에서 다시 할당하는건 당연히 되겠지?? ㅋ

d = 6
def f2(c):
    print(c)
    print(d)
    d = 9

f2(3)

3


UnboundLocalError: local variable 'd' referenced before assignment

In [19]:
# 전역 변수라고 명시적으로 알려줌 
f = 6
def ff():
    f = 10
    def f3(e):
        nonlocal f
        print(e)
        print(f)
        f = 9

f3(3)

3
6


In [20]:
from dis import dis
dis(f1)
print('------------------------------------------------------------------')
dis(f2)
print('------------------------------------------------------------------')
dis(f3)


  5           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  6           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
------------------------------------------------------------------
  5           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (c)
              4 CALL_FUNCTION            1
              6 POP_TOP

  6           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (d)
             12 CALL_FUNCTION            1
             14 POP_TOP

  7          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (d)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
------------------

In [21]:
### 예제 7-8 
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, value):
        self.series.append(value)
        total = sum(self.series)
        return total / len(self.series)

avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [27]:
### 예제 7-9 이동평균을 계산하는 고위 함수

def make_averager():
    series = []
    
    def averager(value):
        series.append(value)
        total = sum(series)
        return total/len(series)
    return averager

avg2 = make_averager()
print(avg2(10))
print(avg2(11))
print(avg2(12))

TypeError: 'total' is an invalid keyword argument for this function

In [23]:
avg2.__code__.co_varnames

('value', 'total')

In [24]:
avg2.__code__.co_freevars

('series',)

In [32]:
avg2.__closure__

(<cell at 0x1063e8d38: list object at 0x106423b88>,)

In [33]:
avg2.__closure__[0].cell_contents

[10]

In [34]:
### 7-13 이력을 유지 안하는 잘 못된 고위 함수 

def make_averager():
    count = 0
    total = 0
    
    def averager(value):
        count += 1
        total += value
        return total / count
    return averager 

avg3 = make_averager()
avg3(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [35]:
### 7-13 이력을 유지 안하는 이동평균 계산 (nonlocal 사용)

def make_averager():
    count = 0
    total = 0
    
    def averager(value):
        nonlocal count, total
        count += 1
        total += value
        return total / count
    return averager 

avg3 = make_averager()
avg3(10)

10.0

In [36]:
avg3(11)

10.5

In [43]:
### 7.16 clock 데커레이터 
import time

def clock(func):
    def clocked(*args):
        """클럭드"""
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ','.join(repr(arg) for arg in args)
        print(f'[{elapsed:.8f}] {name} {arg_str} {result}')
        return result 
    return clocked

In [44]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    """요게 나올까?"""
    return 1 if n < 2 else n * factorial(n - 1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)

print(factorial.__doc__)

print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12787323] snooze 0.123 None
클럭드
**************************************** Calling factorial(6)
[0.00000077] factorial 1 1
[0.00004684] factorial 2 2
[0.00067749] factorial 3 6
[0.00076109] factorial 4 24
[0.00080330] factorial 5 120
[0.00083744] factorial 6 720
6! =  720


In [46]:
### 7-17 개선된 clock 데커레이터 

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_list = []
        
        if args:
            arg_list.append(','.join(repr(arg) for arg in args))
        
        if kwargs:
            paris = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(','.join(paris))
            
        arg_str = ','.join(arg_list)
        print(f'[{elapsed:.8f}] {name}({arg_str}) -> {result}')
        return result
    return clocked
        
    
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)

print(factorial)

print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12462997] snooze(0.123) -> None
<function factorial at 0x104f4e840>
**************************************** Calling factorial(6)
[0.00000095] factorial(1) -> 1
[0.00008607] factorial(2) -> 2
[0.00013590] factorial(3) -> 6
[0.00018001] factorial(4) -> 24
[0.00022197] factorial(5) -> 120
[0.00025702] factorial(6) -> 720
6! =  720


In [ ]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n - 1)

print(fibonacci(6))

In [ ]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n - 1)

print(fibonacci(6))

In [47]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

htmlize({1,2,3})

'<pre>{1, 2, 3}</pre>'

In [48]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [50]:
htmlize('우리집 강아지는 복슬강아지\n')

'<pre>&#x27;우리집 강아지는 복슬강아지\\n&#x27;</pre>'

In [1]:
# 7-21 여러 함수를 범용함수로 묶는 커스텀 html.register() 를 생성하는 singledispatch 
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return f'<p>{content}</p>'

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [2]:
htmlize('우리집 강아지는 복슬강아지\n')

'<p>우리집 강아지는 복슬강아지<br>\n</p>'

In [54]:
print(htmlize(['alpha', 123, {3,2,1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>123 (0x7b)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [55]:
registry = set() 

def register(active=True):
    def decorate(func):
        print(f'레지스터 작동중  active {active} -> decorate ({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('f1() 실행중')
    
    
@register()
def f2():
    print('f2() 실행중')
    
def f3():
    print('f3() 실행중')
    

registry

레지스터 작동중  active False -> decorate (<function f1 at 0x1063f8840>)
레지스터 작동중  active True -> decorate (<function f2 at 0x1063f8d90>)


{<function __main__.f2()>}

In [59]:
# 7-25 매개변수화된 clock() 데커레이터

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name} ({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            print('=====================')
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate


if __name__ == '__main__':
    @clock()
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)
        
    @clock('{name} ({args}) dt={elapsed:0.3f}s')
    def snooze2(seconds):
        time.sleep(seconds)
        
    for i in range(3):
        snooze2(.123)

[0.12310505s] snooze (0.123) -> None
[0.12641311s] snooze (0.123) -> None
[0.12771201s] snooze (0.123) -> None
snooze2 (0.123) dt=0.128s
snooze2 (0.123) dt=0.126s
snooze2 (0.123) dt=0.125s
